## Step 1
Import required libraries

In [ ]:
# This code requires the files; Data and Monthly Tbill to run
# This code requires reading in the tbill and data separtely in this way as it was an earlier task done shortly after the very first task of creating the tangency portfolio
# Later notebooks will use a single raw data file for ease that has different sheet names and so will not run with that data.

import numpy as np
import pandas as pd

## Step 2

Read both files where the return and tbill data is stored (paths go inside " ")

Convert all values to decimal form and annualised rate to monthly rate

In [ ]:
tbilldata = pd.read_excel(
    r"",
    sheet_name="Tbill",
    header=0
)

annualised = tbilldata.iloc[:, 1].to_numpy(dtype=float)
monthlytbill = (1 + annualised / 100) ** (1/12) - 1

assets = pd.read_excel(
    r"",
    sheet_name="Data",
    header=0          
)

assetreturns = assets.iloc[:, 1:].to_numpy(dtype=float) / 100 

## Step 3

Calculate the excess returns find the expectation by finding the mean

Also calculate the covaraince matrix of returns

In [ ]:
excessreturns = assetreturns - monthlytbill.reshape(-1, 1)

T = excessreturns.shape[0]   

expectedexret = excessreturns.sum(axis=0) / T

expectedexret = expectedexret.reshape(-1, 1)

covmat = np.cov(assetreturns, rowvar=False, ddof=1)   

mu = expectedexret
sigma = covmat

## Step 4

Invert the covariance matrix and create a vector of ones

Then use these with the expected excess returns to calculate the weights

Formula := $w_{IS} = \frac{\hat{\sum}^{-1}\hat{\mu}}{\mathbf{1}_N^{\top}\hat{\sum}^{-1}\hat{\mu}}$

In [ ]:
sigmainv = np.linalg.inv(sigma)

onevec = np.ones(len(mu)).reshape(-1, 1)  

mu = mu.reshape(-1, 1)  

numerator = sigmainv @ mu  

denominator = onevec.T @ sigmainv @ mu  

WeightsIS = numerator / denominator

print(WeightsIS)

## Step 5

Calcualte the Sharpe ratio as follows

In [ ]:
rets = assetreturns @ WeightsIS  # (290, 25) @ (25, 1) = (290, 1)
rets = rets.flatten()  # Convert from (290, 1) to (290,)

rf = monthlytbill  

excessreturnz = rets - rf
meanexcess = np.mean(excessreturnz)
stdexcess = np.std(excessreturnz, ddof=1)

sharpe = meanexcess / stdexcess

print(f"Sharpe ratio: {sharpe:.4f}")

## Explanation

**Sharpe Ratio** = 0.6315

Tangency portfolio weights are based off of the whole sample so when calculating the Sharpe ratio we do not have the same estimation error that the portfolios constructed using the rollling window method are subject to.

